# Ejercicio 2: 

Escribe un programa en Python que use Biopython para hacer una búsqueda BLAST de una secuencia de proteína que introduzcas por teclado.
El programa debe guardar en un fichero los resultados que tengan un E-value menor que 0.001. 
El fichero debe contener el identificador, la longitud, el E-value y el porcentaje de identidad de cada resultado. 
Hágalo de forma online y local.

### 2.1. Instalación de BLAST

1. Se descarga la versión para el equipo de BLAST desde el siguiente enlace: [BLAST](https://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST/)
2. Se ejecuta, en nuestro caso, el *.exe* descargado y se instala en el equipo.
3. Se configura el *path* de BLAST como variable de entorno del sistema.
4. Se verifica la instalación con el comando `blastn --version`.

### 2.2. Secuencia de aminoácidos por teclado

In [1]:
from debug_sequence.functions import check_protein

In [2]:
def get_protein():
    protein = input("Enter the protein sequence: ")
    protein = check_protein(protein)
    return protein

### 2.3. Implementación online

Enlace a la documentación: [BLAST Record](https://biopython.org/docs/1.75/api/Bio.Blast.NCBIWWW.html)

Los atributos indicados en el enunciado son:
- Identificador: `alignment.hit_id`
- Longitud: `alignment.length`
- E-value: `hsp.expect`
- Porcentaje de identidad: `hsp.identities / hsp.align_length` (número de identidades / longitud de la secuencia de consulta)


In [3]:
from Bio.Blast import NCBIWWW, NCBIXML

In [27]:
def search_protein_sequence(output_file="./results/blast_results.txt"):

    protein_sequence = get_protein()

    result_handle = NCBIWWW.qblast("blastp", "nr", protein_sequence)
    blast_records = NCBIXML.parse(result_handle)

    with open(output_file, "w") as f:
        for blast_record in blast_records:
            for alignment in blast_record.alignments:
                for hsp in alignment.hsps:
                    if hsp.expect < 0.001:
                            result = {
                                "id": alignment.hit_id,
                                "length": alignment.length,
                                "evalue": hsp.expect,
                                "identity_percentage": (hsp.identities / hsp.align_length) * 100
                            }
                            f.write(str(result) + "\n")
                            
    print(f"Resultados guardados en: {output_file}")

In [28]:
search_protein_sequence()

Resultados guardados en: ./results/blast_results.txt


MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNPKVKAHGKKVL

### 2.4. Implementación local

Para usar las bases de datos proporcionadas por el NCBI, es necesario descargarlas mediante el comando `perl update_blastdb.pl --decompress bbdd`, donde `bbdd` es la base de datos de secuencias de aminoácidos. Las bases de datos utilizadas son las siguientes:

- `swissprot`: repositorio de secuencias de proteínas que forma parte de la base de datos UniProt.



In [29]:
from Bio.Blast.Applications import NcbiblastpCommandline
from Bio.Blast import NCBIXML
import os
import warnings

warnings.simplefilter("ignore")
os.environ["CMD"] = input("Introduce la ruta del ejecutable de BLAST: ")
os.environ["BLASTDB"] = input("Introduce la ruta de la base de datos de BLAST: ")

**CMD** 

C:\Users\Susana\Desktop\blast\blast-2.16.0+\bin\blastp.exe

**DB**

C:\Users\Susana\Desktop\blast\blast-2.16.0+\db\swissprot\swissprot


In [30]:
def search_local_protein(output_file = "./results/blast_results_locally.txt"):
    sequence = input("Introduce la secuencia de proteína en formato FASTA: ")

    fasta_file = "query.fasta"
    with open(fasta_file, "w") as f:
        f.write(">query\n")
        f.write(sequence)

    blastp_cline = NcbiblastpCommandline(
        cmd=os.environ["CMD"],
        query=fasta_file,
        db=os.environ["BLASTDB"],
        evalue=0.001,
        outfmt=5,
        out="results.xml"
    )

    blastp_cline()

    with open("results.xml") as result_handle:
        blast_records = NCBIXML.parse(result_handle)
        filtered_results = []

        for blast_record in blast_records:
            for alignment in blast_record.alignments:
                for hsp in alignment.hsps:
                    if hsp.expect < 0.001:
                        result = {
                            "id": alignment.hit_id,
                            "length": alignment.length,
                            "evalue": hsp.expect,
                            "identity": (hsp.identities / hsp.align_length) * 100
                        }
                        filtered_results.append(result)

    
    with open(output_file, "w") as f:
        f.write("ID\tLength\tE-value\t% Identity\n")
        for res in filtered_results:
            f.write(f"{res['id']}\t{res['length']}\t{res['evalue']:.2e}\t{res['identity']:.2f}%\n")

    print(f"Resultados guardados en {output_file}")

    os.remove(fasta_file)
    os.remove("results.xml")

In [31]:
search_local_protein()

Resultados guardados en ./results/blast_results_locally.txt


MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNPKVKAHGKKVL